In [107]:
import os

from river import datasets
from river import metrics
from river import tree
from river import ensemble
from river import evaluate
from river import compose
from river import naive_bayes
from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
from confluent_kafka import Producer,Consumer
import certifi
import time
import json
import pandas

In [108]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [109]:
feature_topic = 't_8'

In [112]:
classifiers=[]
classifier='HoeffdingAdaptiveTreeClassifier'
classifiers.append(classifier)
classifier='SRPClassifierHAT'
classifiers.append(classifier)
classifier='SRPClassifierNaiveBayes'
classifiers.append(classifier)
classifier='AdaptiveRandomForestClassifier'
classifiers.append(classifier)
classifier='HalfSpaceTrees'
classifiers.append(classifier)

model_artifact = None

In [113]:
max_size=1000
dataset = datasets.MaliciousURL()
d=[]
for classifier in classifiers:
    data = dataset.take(max_size)
    if classifier=='HoeffdingAdaptiveTreeClassifier':
            model_artifact = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10,seed=0)
    elif classifier=='SRPClassifierHAT':
            model_artifact =  ensemble.SRPClassifier(
                                                   model=tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10, seed=0), seed=42,
                                                )
    elif classifier=='SRPClassifierNaiveBayes':
            model_artifact = ensemble.SRPClassifier(
                                                 model=naive_bayes.BernoulliNB(alpha=1), seed=42,
                                              )
    elif classifier=='AdaptiveRandomForestClassifier':
            model_artifact = ensemble.AdaptiveRandomForestClassifier(leaf_prediction="mc")

    elif classifier=='HalfSpaceTrees':
            model_artifact = compose.Pipeline(preprocessing.MinMaxScaler(),anomaly.HalfSpaceTrees(seed=42))
    cnt = 0
    training_st_ts = time.time()
    for f, y in data:
        cnt = cnt + 1
        model_artifact = model_artifact.learn_one(f,y)
    training_end_ts = time.time()
    total_ts = training_end_ts-training_st_ts
    avg_ts = (total_ts/max_size) * 1000    
    d.append({'Classifier':classifier,
             'Avg Time(ms)': (total_ts/max_size) * 1000,
             'Exp Train Time 1 million records (s)':avg_ts * 1000,
             'Exp time 1 million (mins)': avg_ts * (1000/60)})
df = pandas.DataFrame(data=d)
display(df)

,Classifier,Avg Time(ms),Exp Train Time 1 million records (s),Exp time 1 million (mins)
0,HoeffdingAdaptiveTreeClassifier,1.919338,1919.337511,31.988959
1,SRPClassifierHAT,29.139747,29139.746666,485.662444
2,SRPClassifierNaiveBayes,4.398058,4398.058414,73.300974
3,AdaptiveRandomForestClassifier,1.029698,1029.698372,17.161640
4,HalfSpaceTrees,0.229605,229.604959,3.826749


In [ ]:
cnt = 0
training_st_ts = time.time()
for f, y in data:
    cnt = cnt + 1
    model_artifact = model_artifact.learn_one(f,y)
training_end_ts = time.time()
total_ts = training_end_ts-training_st_ts
avg_ts = (total_ts/max_size) * 1000
print(f'Training using no of records - {cnt}, total time taken in seconds- {total_ts} ')        
print(f'Training using no of records - {cnt}, avg time per record in ms - {avg_ts} ')     
print(f'Expected time to train using 1 million records {avg_ts * 1000} seconds or {avg_ts * 1000/60} minutes')


In [ ]:
total_records=1000000
durations_per_partition = pandas.DataFrame()
print(f'Total number of records - 1 million')
no_of_partitions = range(10,201,10)
d=[]
for p in no_of_partitions:
    d.append({'No of partitions':p,'Records per partition':round(total_records/p),'Training time(s)':(avg_ts * 1000/p),'Training time(mins)':(avg_ts * 1000/(60*p))})
df = pandas.DataFrame(data=d)
display(df)

In [78]:
data = dataset.take(max_size)
cnt = 0
prediction_st_ts = time.time()
for f, y in data:
    cnt = cnt + 1
    model_artifact.predict_one(f)
prediction_end_ts = time.time()
total_ts = prediction_end_ts-prediction_st_ts
avg_ts = (total_ts/max_size) * 1000
print(f'Prediction using no of records - {cnt}, total time taken in seconds- {total_ts} ')        
print(f'Prediction using no of records - {cnt}, avg time per record in ms - {avg_ts} ')     
print(f'Expected time to predict using 1 million records {avg_ts * 1000} seconds or {avg_ts * 1000/60} minutes')


Prediction using no of records - 10000, total time taken in seconds- 1.8953659534454346 
Prediction using no of records - 10000, avg time per record in ms - 0.18953659534454345 
Expected time to predict using 1 million records 189.53659534454346 seconds or 3.158943255742391 minutes


In [79]:
total_records=1000000
durations_per_partition = pandas.DataFrame()
print(f'Total number of records - 1 million')
no_of_partitions = range(10,201,10)
d=[]
for p in no_of_partitions:
    d.append({'No of partitions':p,'Records per partition':round(total_records/p),'Prediction time(s)':(avg_ts * 1000/p),'Prediction time(mins)':(avg_ts * 1000/(60*p))})
df = pandas.DataFrame(data=d)
display(df)

Total number of records - 1 million


,No of partitions,Records per partition,Prediction time(s),Prediction time(mins)
0,10,100000,18.953660,0.315894
1,20,50000,9.476830,0.157947
2,30,33333,6.317887,0.105298
3,40,25000,4.738415,0.078974
4,50,20000,3.790732,0.063179
5,60,16667,3.158943,0.052649
6,70,14286,2.707666,0.045128
7,80,12500,2.369207,0.039487
8,90,11111,2.105962,0.035099
9,100,10000,1.895366,0.031589


In [18]:
def consume_messages(group_id,model,only_predict=False):
    global model_artifact
    features_consumer_conf = {'bootstrap.servers': bsts,
                          'sasl.username': user,
                          'sasl.password': password,
                          'sasl.mechanism': 'PLAIN',
                          'security.protocol': 'SASL_SSL',
                          'ssl.ca.location': certifi.where(),
                          'group.id': group_id,
                          'enable.auto.commit': True,
                          'auto.commit.interval.ms':1000,         
                          'auto.offset.reset': 'latest'}
    features_consumer = Consumer(features_consumer_conf)  
    
    print(f'\nNow subscribing to features topic:{feature_topic}')
        
    features_consumer.subscribe([feature_topic])
    cnt = 0
    msg = None
    error_cnt = 0
    end_learn_ts = 0
    st_learn_ts = 0

    st_processing_time = 0
    
    learning_durations=[]
    prediction_durations=[]
    processing_durations = []
    score_and_truth = []
    mem_usage = []
    while(True):           
        msg = features_consumer.poll(timeout=0.1)    
        if msg is None: continue
        if msg.error():
            error_cnt = error_cnt + 1
            if msg.error().code() == KafkaError._PARTITION_EOF:                    
                    if(error_cnt%1000==0):
                        print('error')
                        print(msg)
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                             (msg.topic(), msg.partition(), msg.offset()))
        else:       
            try:         
                msg_arrival_time = time.time()
                message = json.loads(msg.value().decode("utf-8"))            
                cnt = cnt + 1
                
                f = message['f']
                y = (message['y']=='true')              
                if(cnt==1):
                    st_processing_time = time.time()
                
                st_prediction_time = time.time()            
                score = model_artifact.predict_one(f)
                score_and_truth.append({'y':y,'score':score})
                end_prediction_time = time.time()  
                prediction_durations.append(end_prediction_time-st_prediction_time)
                
                if not only_predict:
                    st_learn_ts = time.time()
                    model_artifact = model_artifact.learn_one(f,y)      
                    end_learn_ts = time.time()
                    learning_durations.append(end_learn_ts-st_learn_ts)            
                
                msg_departure_time = time.time()
                processing_durations.append(msg_departure_time-msg_arrival_time)
                if(cnt%10==0):
                    mem_usage.append(model_artifact._raw_memory_usage)
            except Exception as  e:      
                print(json.loads(msg.value().decode("utf-8")))
                print(e, file=sys.stdout)
                ignored = ignored + 1
                print(f'ignored ={ignored} total = {cnt}')

    print('CLOSING')
    features_consumer.commit()
    features_consumer.close() 
    total_time = time.time() - st_processing_time
    return 

In [19]:
import statistics
from river import metrics

def print_time_results(durations, type_of_duration):
    if len(durations)==0:
        return
    mean = statistics.mean(durations)
    median = statistics.median(durations)
    max_dur = max(durations)
    min_dur = min(durations)  
    print(f'Type of durations : {type_of_duration} ' )
    print(f'\tAVG : {mean}')
    print(f'\MEDIAN : {median}')
    print(f'\MAX : {max_dur}')
    print(f'\MIN : {min_dur}')
    
def print_results(score_and_truth,processing_durations, prediction_durations, learning_durations,mem_usage,total_time):
    auc = metrics.ROCAUC()
    f1 = metrics.F1()
    recall = metrics.MicroRecall()
    for m in score:
        y = m['y']
        score = m['score']
        auc = auc.update(y,score)
        f1 = f1.update(y, score)
        recall = recall.update(y, score)
    
  
    total_records = len(durations)
    avg_memory_usage = statistics.mean(mem_usage)
    print(f'Messages consumed:{total_records},Total Cumulative Time: {total_time}')    
    print(f'AUC{auc}')
    print(f'F1 {f1}')
    print(f'RECALL {recall}')
    print(f'AVERAGE MEMORY USAGE {avg_memory_usage}')
    print_time_results(processing_durations,f'PROCESSING DURATIONS FOR {processing_durations}')
    print_time_results(prediction_durations,f'PREDICTION DURATIONS FOR {prediction_durations}')
    print_time_results(prediction_durations,f'LEARNING DURATIONS FOR {learning_durations}')



In [ ]:

group_id = 'ARFC_8'
only_predict=False
score_and_truth,processing_durations, prediction_durations, learning_durations,mem_usage,total_time = consume_messages(group_id,model,only_predict)
print_results(score_and_truth,processing_durations, prediction_durations, learning_durations,mem_usage,total_time)


Now subscribing to features topic:test_2


In [ ]:
only_predict=True
score_and_truth,processing_durations, prediction_durations, learning_durations,mem_usage,total_time = consume_messages(group_id,model,only_predict)
print_results(score_and_truth,processing_durations, prediction_durations, learning_durations,mem_usage,total_time)